In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math as mth
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

print("sample")
print(np.isclose(10,9, rtol=0.12))

df_train=pd.read_csv("../input/train.csv", sep=',', lineterminator='\n')
df=df_train[(df_train.target==1)]
print(df.index)

#list of columns in dataframe
dynamic_list=df.columns

def check_fix_Null(d,a):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for i in range(len(a)):
        if d[d[a].columns[i]].isnull().sum() >0 and d[d[a].columns[i]].dtype not in numerics:
            d[d[a].columns[i]]=d[d[a].columns[i]].fillna(value='NO VALUE')
        elif  d[d[a].columns[i]].isnull().sum() >0 and d[d[a].columns[i]].dtype  in numerics:
            d[d[a].columns[i]]=d[d[a].columns[i]].fillna(value=0)

check_fix_Null(df,dynamic_list)


def ListNon_N(d,a):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    d['new_column']=0   
    for i in range(len(a)):
            print(i)
            if d[d[a].columns[i]].dtype not in numerics:
                print(d[a].columns[i])
                #print(d[d[a].columns[i]].unique())
                print(d[d[a].columns[i]].apply(hash).unique())
                if i==0:
                    d['new_column']=d[d[a].columns[i]].apply(hash)
                else:
                    d['new_column']=d[d[a].columns[i]].apply(hash)+d['new_column']
                
print(ListNon_N(df,dynamic_list))



tmp_val=0.1
def corr_check(dt,a):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    dt["final_value"]=""
    d=dt[(dt.target==1)] # taking dataset which is target=1
    for i in range(len(a)):
            if d[d[a].columns[i]].dtype  in numerics and d[a].columns[i]!='ID'  and d[a].columns[i]!='target':
                print(d[a].columns[i])
                print(d[d[a].columns[i]].quantile(1))
                #col_name=d[a].columns[i]+"_v"
                col_name_final="final_value"#d[a].columns[i]+"_f"
                dict = {True: "1", False: "0"}
                #d["col_name1"]=np.isclose(d[d[a].columns[i]],d[d[a].columns[i]].mean(), rtol=tmp_val)
                d["col_name2"]=np.isclose(d[d[a].columns[i]],d[d[a].columns[i]].quantile(1), rtol=tmp_val)
                d["col_name3"]=np.isclose(d[d[a].columns[i]],d[d[a].columns[i]].quantile(0.75), rtol=tmp_val)
                d["col_name4"]=np.isclose(d[d[a].columns[i]],d[d[a].columns[i]].quantile(0.5), rtol=tmp_val)
                d["col_name5"]=np.isclose(d[d[a].columns[i]],d[d[a].columns[i]].quantile(0.25), rtol=tmp_val)
                d["col_name_final_tmp"]=d["col_name2"].map(dict)+d["col_name3"].map(dict)+d["col_name4"].map(dict)+d["col_name5"].map(dict)
                if i==1:
                    d[col_name_final]=d["col_name_final_tmp"]
                elif i>1:
                    d[col_name_final]=d[col_name_final]+d["col_name_final_tmp"]
                    #print("xz"+d["col_name1"])
                    print(col_name_final)
                    print(d[col_name_final].dtype)
                    print(d[ [ d[a].columns[i],col_name_final  ]  ])
    dt["final_value"]=d[col_name_final]

corr_check(df,dynamic_list)

print(df)

var_mod = ['final_value']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df.dtypes 

predictor_var = df[['new_column','final_value']]
outcome_var = df['target']
model = XGBClassifier()
model.fit(predictor_var,outcome_var)
predictions = model.predict(predictor_var)
accuracy = metrics.accuracy_score(predictions,outcome_var)
print("Accuracy : %s" % "{0:.3%}".format(accuracy) )


predictors=['new_column','final_value']
outcome='target'



train_predictions = model.predict_proba(predictor_var)
print("predictions")
print(train_predictions)
print(outcome_var)
ww = log_loss(outcome_var, train_predictions)
print("Log Loss: {}".format(w))



#Perform k-fold cross-validation with 5 foldshttps://www.kaggle.com/ozgurb
kf = KFold(df.shape[0], n_folds=5)
error = []

for train, test in kf:
    # Filter training data
    train_predictors = (df[predictors].iloc[train,:])
    # The target we're using to train the algorithm.
    train_target = df[outcome].iloc[train]
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    #Record error from each cross-validation run
    error.append(model.score(df[predictors].iloc[test,:], df[outcome].iloc[test]))

print( "Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)) )

